# SSM Challenge
* StelllarAlgo Data Science
* Peter Morrison
* June 1, 2022

Environment: Explore-US

SSM Paramter we are reading: `/customer/model-retention/ai/db-connections/data-sci-retention/database-read`

This challenge is to:
1. Read the value of the SSM Paramter
2. Use the value in the SSM parameter to form a connection string to MSSQL
3. Read the contents of `ds.customerScores` into a dataframe

In [29]:
import awswrangler as wr
import boto3 
import json
import pandas as pd
import pyodbc

from IPython.display import display

In [2]:
! aws sso login --profile Explore-US-DataScienceAdmin

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

ZCTS-ZXJJ
Successfully logged into Start URL: https://stellaralgo.awsapps.com/start


In [3]:
session = boto3.setup_default_session(profile_name='Explore-US-DataScienceAdmin')
client = boto3.client('ssm')

## Read Param from SSM
This section is reading the param to a variable

In [34]:
PARAMETER_NAME = '/customer/model-retention/ai/db-connections/data-sci-retention/database-read'
connection_string = client.get_parameter(Name=PARAMETER_NAME, WithDecryption=True)

## Form Connection to MSSQL w/ Connection String from Param
This section is forming the connection.

In [35]:
# the Value info comes through as a string so we need to load it to a dict here
connection_values = json.loads(connection_string['Parameter']['Value'])

In [36]:
# The parameter didn't send the right IP so the correct one must be placed below
SERVER = '34.206.73.189'
PORT = connection_values['port']
DATABASE = connection_values['database']
USERNAME = connection_values['username']
PASSWORD = connection_values['password']
TABLE = connection_values['name']
ODBC = '{ODBC Driver 17 for SQL Server}'


connection_string = f'DRIVER={ODBC};SERVER={SERVER},{PORT};DATABASE={DATABASE};UID={USERNAME};PWD={PASSWORD}'


conn = pyodbc.connect(connection_string)
df = pd.read_sql_query(f'select top(100) * from ds.customerScores', conn)
display(df)
conn.close()

/usr/local/lib/python3.8/dist-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,dimcustomermasterid,buyer_score,tenuredays,attendancePercentage,mostRecentAttendance,year,lkupclientid,productGrouping,seasonYear,insertDate
0,349335093,0.9115,2190,0.555556,2022-01-17,2021,33,Full Season,2021,01-22-2022 09:33:41
1,349335290,0.3554,1825,1.037037,2022-01-17,2021,33,Full Season,2021,01-22-2022 09:33:41
2,349335440,0.3491,1825,1.142857,2022-01-17,2021,33,Partial,2021,01-22-2022 09:33:41
3,349335508,0.359,2190,1.037037,2022-01-17,2021,33,Full Season,2021,01-22-2022 09:33:41
4,349344774,0.8677,1460,0.814815,2022-01-17,2021,33,Full Season,2021,01-22-2022 09:33:41
...,...,...,...,...,...,...,...,...,...,...
95,352986413,0.414,1095,1.000000,2022-01-17,2021,33,Full Season,2021,01-22-2022 09:33:41
96,352986684,0.6225,1460,1.000000,2022-01-06,2021,33,Partial,2021,01-22-2022 09:33:41
97,352987192,0.5432,365,0.851852,2022-01-17,2021,33,Full Season,2021,01-22-2022 09:33:41
98,352987363,0.4904,2190,0.666667,2021-12-12,2021,33,Partial,2021,01-22-2022 09:33:41
